### //TODO
- tune classifier params
- confusion matrix
- add logistic regression

In [39]:
import itertools
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from pandas.tools.plotting import table

In [40]:
df=pd.read_csv("ClustersCountiesVoting.csv").drop('Unnamed: 0',axis=1)

In [41]:
for i in ['Current_Cluster',
       'Demographic_Cluster', 'Economic_Cluster', 'Owned_Cluster',
       'Age_Cluster', 'Education_Cluster', 'Housing_Cluster','Combined_Cluster']:
    cols=pd.get_dummies(df[i])
    strings=[]
    for j in range(len(cols.columns)):
        strings.append(i+str(j))
        df[i+str(j)]=cols[cols.columns[j]]
    print(i,len(cols.columns))
    cols.columns=strings

Current_Cluster 6
Demographic_Cluster 7
Economic_Cluster 9
Owned_Cluster 7
Age_Cluster 5
Education_Cluster 6
Housing_Cluster 7
Combined_Cluster 10


In [42]:
df['Winner2016']=0
df.ix[df['per_point_diff_2016']>0,'Winner2016']=1
df['Winner2012']=0
df.ix[df['per_point_diff_2012']>0,'Winner2012']=1
dems=df[df['Winner2016']==1]

In [43]:
demographic=["Demographic_Cluster0","Demographic_Cluster1","Demographic_Cluster2","Demographic_Cluster3",
            "Demographic_Cluster4","Demographic_Cluster5","Demographic_Cluster6"]
economic=["Economic_Cluster0","Economic_Cluster1","Economic_Cluster2","Economic_Cluster3",
            "Economic_Cluster4","Economic_Cluster5","Economic_Cluster6","Economic_Cluster7","Economic_Cluster8"]
econ=["Economic_Cluster0"]
education=["Education_Cluster0","Education_Cluster1","Education_Cluster2","Education_Cluster3","Education_Cluster4",
                "Education_Cluster5"]

## Classification algos & tuning

In [44]:
def mean_f_score(truth, pred):
    return precision_recall_fscore_support(truth, pred, average='micro')[2]

In [45]:
def get_classifiers(X, Y):
    models = []
    
    svc = SVC()
    svc.fit(X, Y)
    models.append(svc)
    
    knn = KNeighborsClassifier(n_jobs=-1, weights='distance')
    knn.fit(X, Y)
    models.append(knn)
    
    logit = LogisticRegression(n_jobs=-1)
    logit.fit(X,Y)
    models.append(logit)
    
    return models

In [46]:
def get_predictor_strings():
    pre = ["svc", "knn", "logit"]
    yr =  ["2012", "2016"]
    suf = ["all", "subset"]
    return [p + "_" + y + "_" + s for s in suf for y in yr for p in pre]


In [47]:
def clusteringalgo_fscore(x, y, y2, df, df2):
    names = get_predictor_strings()
    predictions = [] * len(names)
    
    frames = [df, df, df2, df2]
    C = get_classifiers(df[x], df[y])
    for d in frames:
        for c in C:
            predictions.append(c.predict(d[x]))
            
    out = []
    for i in range(len(names)):
        k = names[i]
        if "2012" in k:
            if "all" in k:
                out.append((k, mean_f_score(df[y], predictions[i]), confusion_matrix(df[y], predictions[i])))
            else:
                out.append((k, mean_f_score(df2[y], predictions[i]), confusion_matrix(df2[y], predictions[i])))
        else:
            if "all" in k:
                out.append((k, mean_f_score(df[y2], predictions[i]), confusion_matrix(df[y2], predictions[i])))
            else:
                out.append((k, mean_f_score(df2[y2], predictions[i]), confusion_matrix(df2[y2], predictions[i])))

    return out

In [48]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    plt.show()
# Plot normalized confusion matrix



In [49]:

def plot_conf_mat(results, title="Confusion Matrix"):
    run, score, mats = zip(*results)
    for i in range(len(results)):
        f, arr = plt.subplots(len(results))
        arr = plot_confusion_matrix(mats[i], normalize=True, title=title, "")
        f.show()

SyntaxError: positional argument follows keyword argument (<ipython-input-49-9875e60f8c8d>, line 6)

In [50]:
def plot_scores(info:list):
    labels, scores, cf_mats = zip(*info)
    keys=["2012_all","2012_subset","2016_all","2016_subset"]
    num_keys = len(keys)
    pred_by_c = {"svc":[0]*(num_keys), "knn":[0]*(num_keys), "logit":[0]*(num_keys)}
    data = {k : [0] * (num_keys-1) for k in keys}
    data["classifier"] = [None]*3
    
    for i in range(len(labels)):
        L = labels[i]
        if "svc" in L:
            c = "svc"
        elif "knn" in L:
            c = "knn"
        else:
            c = "logit"
        
        for k in range(num_keys):
            if keys[k] in L:
                pred_by_c[c][k] = scores[i]
                
    c_idx = 0 
    print(pred_by_c.keys())
    for c in pred_by_c.keys():
        s = pred_by_c[c]
        for i in range(num_keys):
            data[keys[i]][c_idx] = s[i]
            data["classifier"][c_idx] = c
        c_idx+=1
            
    cols = keys.copy()
    cols.append("classifier")
    return pd.DataFrame(data=data, columns=cols)
    
    
        
    

## SWING STATES

In [51]:
swing_states=["AZ","FL","GA","NH","NC","CO","MI","NV","VA","PA","WI","OH"]

In [52]:
swingers=df[df['state_abbreviation'].isin(swing_states)]

In [54]:
two = clusteringalgo_fscore(["Female%",
                     "White%","Black%","NativeA%","Asian%",
                    "PIsland%","TwoRace%","Latino", "WhiteNLat%"],"Winner2012","Winner2016",swingers,swingers[swingers['Demographic_Cluster1']==1])

ax = plt.subplot(111, frame_on=False) # no visible frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis

table(ax, plot_scores(two))  # where df is your data frame

plt.savefig('f_score_race_demog1.png')

dict_keys(['knn', 'logit', 'svc'])


In [32]:
three = clusteringalgo_fscore(["Female%",
                     "White%","Black%","NativeA%","Asian%",
ax = plt.subplot(111, frame_on=False) # no visible frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis

table(ax, plot_scores(three))  # where df is your data frame

plt.savefig('mytable.png')              

dict_keys(['knn', 'logit', 'svc'])


,2012_all,2012_subset,2016_all,2016_subset,classifier
0,1.000000,1.000000,0.904000,0.888889,knn
1,0.825143,0.777778,0.896000,0.888889,logit
2,0.915429,1.000000,0.926857,0.888889,svc


In [33]:
four = clusteringalgo_fscore(['Female%',
 'White%',
 'Black%',
 'NativeA%',
 'Asian%',
 'PIsland%',
 'TwoRace%',
 'Latino',
 'WhiteNLat%',
 'PerCapitaIncome',
 'MedianIncome',
 '%BelowPoverty'],"Winner2012","Winner2016",swingers,swingers[swingers['Combined_Cluster0']==1])
ax = plt.subplot(111, frame_on=False) # no visible frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis

table(ax, plot_scores(four))  # where df is your data frame

plt.savefig('f_score_combined0.png') 

dict_keys(['knn', 'logit', 'svc'])


,2012_all,2012_subset,2016_all,2016_subset,classifier
0,1.000000,1.000000,0.904000,0.876712,knn
1,0.822857,0.890411,0.886857,0.876712,logit
2,1.000000,1.000000,0.904000,0.876712,svc


In [34]:
five = clusteringalgo_fscore(['Female%',
 'White%',
 'Black%',
 'NativeA%',
 'Asian%',
 'PIsland%',
 'TwoRace%',
 'Latino',
 'WhiteNLat%',
 'PerCapitaIncome',
 'MedianIncome',
 '%BelowPoverty'],"Winner2012","Winner2016",swingers,swingers[swingers['Combined_Cluster4']==1])
plot_scores(five)

dict_keys(['knn', 'logit', 'svc'])


,2012_all,2012_subset,2016_all,2016_subset,classifier
0,1.000000,1.000000,0.904000,0.864198,knn
1,0.822857,0.740741,0.886857,0.851852,logit
2,1.000000,1.000000,0.904000,0.864198,svc


In [35]:
six = clusteringalgo_fscore(['Female%',
 'White%',
 'Black%',
 'NativeA%',
 'Asian%',
 'PIsland%',
 'TwoRace%',
 'Latino',
 'WhiteNLat%',
 'PerCapitaIncome',
 'MedianIncome',
 '%BelowPoverty'],"Winner2012","Winner2016",swingers,swingers[swingers['Combined_Cluster5']==1])
plot_scores(six)

dict_keys(['knn', 'logit', 'svc'])


,2012_all,2012_subset,2016_all,2016_subset,classifier
0,1.000000,1.00000,0.904000,0.918919,knn
1,0.822857,0.72973,0.886857,0.810811,logit
2,1.000000,1.00000,0.904000,0.918919,svc


In [36]:
|

SyntaxError: invalid syntax (<ipython-input-36-6f719179e65e>, line 1)